In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import (
    confusion_matrix, classification_report, precision_recall_curve,
    f1_score, precision_score, recall_score, average_precision_score
)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (64, 64))
            images.append(img)
            labels.append(label)
    return images, labels

class_names = [
    'black_king', 'black_queen', 'black_bishop', 'black_knight', 'black_rook', 'black_pawn',
    'white_king', 'white_queen', 'white_bishop', 'white_knight', 'white_rook', 'white_pawn'
]

X, y = [], []
for class_name in class_names:
    folder = f'data/train/{class_name}'
    imgs, labels = load_images_from_folder(folder, class_name)
    X.extend(imgs)
    y.extend(labels)

X = np.array(X)
y = np.array(y)

le = LabelEncoder()
y_encoded = le.fit_transform(y)

def extract_hog_features(images):
    features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        feat = hog(
            gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
            visualize=False, multichannel=False
        )
        features.append(feat)
    return np.array(features)

X_features = extract_hog_features(X)

X_train, X_val, y_train, y_val = train_test_split(
    X_features, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

clf = SVC(kernel='linear', probability=True, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)
y_proba = clf.predict_proba(X_val)

cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig('results/confusion_matrix.png')
plt.show()

print("Classification Report:")
print(classification_report(y_val, y_pred, target_names=le.classes_))

f1_scores = f1_score(y_val, y_pred, average=None)
f1_macro = f1_score(y_val, y_pred, average='macro')
plt.figure(figsize=(10, 6))
plt.bar(le.classes_, f1_scores)
plt.title('F1 Score per Class')
plt.ylabel('F1 Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('results/f1_curve.png')
plt.show()
print(f"Macro F1 Score: {f1_macro:.2f}")

precision = dict()
recall = dict()
average_precision = dict()
for i in range(len(class_names)):
    precision[i], recall[i], _ = precision_recall_curve(
        (y_val == i).astype(int), y_proba[:, i]
    )
    average_precision[i] = average_precision_score((y_val == i).astype(int), y_proba[:, i])

plt.figure(figsize=(12, 8))
for i in range(len(class_names)):
    plt.plot(recall[i], precision[i], lw=2, label=f'{le.classes_[i]} (AP={average_precision[i]:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')
plt.tight_layout()
plt.savefig('results/pr_curve.png')
plt.show()

macro_precision = precision_score(y_val, y_pred, average='macro')
macro_recall = recall_score(y_val, y_pred, average='macro')
print(f"Macro Precision: {macro_precision:.2f}")
print(f"Macro Recall: {macro_recall:.2f}")

plt.figure(figsize=(10, 6))
for i in range(len(class_names)):
    plt.plot(recall[i], precision[i], label=le.classes_[i])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Curves per Class')
plt.legend()
plt.tight_layout()
plt.savefig('results/precision_curve.png')
plt.show()
